In [0]:
# Load from Databricks workspace tables
bookings = spark.table("workspace.default.bookings")
customers = spark.table("workspace.default.customers")

# Register as SQL views (for SQL queries later)
bookings.createOrReplaceTempView("Bookings")
customers.createOrReplaceTempView("Customers")

In [0]:
from pyspark.sql.functions import col, count

popular_services = (bookings
    .groupBy("Service")
    .agg(count("*").alias("Total_Bookings"))
    .orderBy(col("Total_Bookings").desc())
)

popular_services.show()
popular_services.display()

+-------------+--------------+
|      Service|Total_Bookings|
+-------------+--------------+
|      Massage|            18|
|Hair Coloring|            15|
|       Facial|            14|
|  Spa Therapy|            13|
|   Beard Trim|            12|
|     Manicure|            10|
|     Pedicure|            10|
|      Haircut|             8|
+-------------+--------------+



Service,Total_Bookings
Massage,18
Hair Coloring,15
Facial,14
Spa Therapy,13
Beard Trim,12
Manicure,10
Pedicure,10
Haircut,8


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import hour

peak_times = (bookings
    .withColumn("Hour", hour(col("Time")))
    .groupBy("Hour")
    .agg(count("*").alias("Bookings_Count"))
    .orderBy(col("Bookings_Count").desc())
)

peak_times.show()
peak_times.display()

+----+--------------+
|Hour|Bookings_Count|
+----+--------------+
|  17|            13|
|  13|            10|
|   9|            10|
|  10|            10|
|  19|             9|
|  15|             9|
|  20|             9|
|  12|             8|
|  14|             8|
|  18|             7|
|  16|             6|
|  11|             1|
+----+--------------+



Hour,Bookings_Count
17,13
13,10
9,10
10,10
19,9
15,9
20,9
12,8
14,8
18,7


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT Service, 
       SUM(Amount) AS Total_Revenue, 
       AVG(Amount) AS Avg_Revenue_Per_Booking
FROM workspace.default.bookings
GROUP BY Service
ORDER BY Total_Revenue DESC;


Service,Total_Revenue,Avg_Revenue_Per_Booking
Massage,34607,1922.611111111111
Hair Coloring,25647,1709.8
Spa Therapy,22029,1694.5384615384614
Facial,21555,1539.642857142857
Beard Trim,17831,1485.9166666666667
Manicure,16420,1642.0
Pedicure,15372,1537.2
Haircut,10125,1265.625


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT c.Customer_ID, c.Name, COUNT(b.Booking_ID) AS Total_Visits
FROM workspace.default.customers c
JOIN workspace.default.bookings b 
  ON c.Customer_ID = b.Customer_ID
GROUP BY c.Customer_ID, c.Name
HAVING COUNT(b.Booking_ID) > 1
ORDER BY Total_Visits DESC;


Customer_ID,Name,Total_Visits
CUST033,Riya,8
CUST036,Saanvi,5
CUST015,Kavya,5
CUST028,Kavya,5
CUST014,Saanvi,5
CUST005,Arjun,4
CUST001,Riya,4
CUST016,Arjun,4
CUST035,Aadhya,4
CUST008,Kavya,3


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT Staff_ID, 
       COUNT(*) AS Total_Services, 
       SUM(Amount) AS Total_Revenue
FROM workspace.default.bookings
GROUP BY Staff_ID
ORDER BY Total_Revenue DESC;

Staff_ID,Total_Services,Total_Revenue
STF09,12,21283
STF10,12,20680
STF05,11,19889
STF02,12,18237
STF04,12,17969
STF03,12,17731
STF06,11,17600
STF08,8,15620
STF07,5,9795
STF01,5,4782


Databricks visualization. Run in Databricks to view.

In [0]:
# Gender-based spending
gender_spend = (bookings.join(customers, "Customer_ID")
    .groupBy("Gender")
    .agg({"Amount":"sum", "Amount":"avg"})
)
gender_spend.show()
gender_spend.display()



+------+------------------+
|Gender|       avg(Amount)|
+------+------------------+
|Female|1725.8431372549019|
|  Male| 1542.204081632653|
+------+------------------+



Gender,avg(Amount)
Female,1725.8431372549019
Male,1542.204081632653


Databricks visualization. Run in Databricks to view.